<div class="alert alert-block alert-info">
    <center><h1>ELT Project Overview: GoodReads Data</h1></center>
</div>

In [11]:
import pandas as pd
import csv
from sqlalchemy import create_engine
import numpy as np

<div class="alert alert-block alert-info">
    <center><h1>CSV File Import to Dataframe</h1></center>
</div>

In [12]:
csv_file = "books.csv"
books_df = pd.read_csv((csv_file), dtype={'authors': str}, error_bad_lines=False)
books_df.head()

b'Skipping line 4012: expected 10 fields, saw 11\nSkipping line 5688: expected 10 fields, saw 11\nSkipping line 7056: expected 10 fields, saw 11\nSkipping line 10601: expected 10 fields, saw 11\nSkipping line 10668: expected 10 fields, saw 11\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964


<div class="alert alert-block alert-info">
    <h1>Dropping Columns ~ Selecting Columns Needed for Database</h1>
</div>

In [13]:
books = books_df[['title', 'authors', 'average_rating', 'isbn', 'ratings_count']].copy()
books.head()

,title,authors,average_rating,isbn,ratings_count
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,1944099
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,1996446
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,5629932
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,6267
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,2149872


In [14]:
#Checking dataframe columns to make sure data is clean -- no extra rows or missing information

books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13714 entries, 0 to 13713
Data columns (total 5 columns):
title             13714 non-null object
authors           13714 non-null object
average_rating    13714 non-null float64
isbn              13714 non-null object
ratings_count     13714 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 535.8+ KB


In [15]:
#Changing dataframe data types for columns to string so name can be separated
books.authors = books.authors.astype(str)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13714 entries, 0 to 13713
Data columns (total 5 columns):
title             13714 non-null object
authors           13714 non-null object
average_rating    13714 non-null float64
isbn              13714 non-null object
ratings_count     13714 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 535.8+ KB


<div class="alert alert-block alert-info">
    <center><h1>Renaming Columns for Database</h1>
    * Coordinating naming conventions for columns based on shared CSV files for easier querying/analysis</center>
</div>


In [16]:
# Title, AuthorLast, AuthorFirst, ISBN, Rating, Rating Count, Price ($) -- my data does not include price

#Rename Columns
books = books.rename(columns={"title":"title", "authors": "authors", "average_rating": "goodreads_rating", "isbn": "isbn10", "ratings_count": "goodreads_ratings_count"})
books.head()

,title,authors,goodreads_rating,isbn10,goodreads_ratings_count
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,1944099
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,1996446
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,5629932
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,6267
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,2149872


<div class="alert alert-block alert-info">
    <h1>Splitting and Cleaning Authors' Column into Last Name and First Name Columns</h1>
</div>

In [17]:
# dropping null value columns to avoid errors 
books.dropna(inplace = True) 
  
# new data frame with split value columns - first pass to separate out multiple authors
new = books["authors"].str.split("-", n = 1, expand = True) 
  
# making separate author columns from new data frame 
books["first_author"]= new[0] 
  
# making separate author columns from new data frame 
books["second_author"]= new[1] 
  
# Dropping old Name columns 
books.drop(columns =["authors"], inplace = True) 


# df display 
books.head()

,title,goodreads_rating,isbn10,goodreads_ratings_count,first_author,second_author
0,Harry Potter and the Half-Blood Prince (Harry ...,4.56,0439785960,1944099,J.K. Rowling,Mary GrandPré
1,Harry Potter and the Order of the Phoenix (Har...,4.49,0439358078,1996446,J.K. Rowling,Mary GrandPré
2,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,0439554934,5629932,J.K. Rowling,Mary GrandPré
3,Harry Potter and the Chamber of Secrets (Harry...,4.41,0439554896,6267,J.K. Rowling,None
4,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,043965548X,2149872,J.K. Rowling,Mary GrandPré


In [18]:
books.drop(columns = ["second_author"], inplace = True)
  
# df display 
books.head(20)

,title,goodreads_rating,isbn10,goodreads_ratings_count,first_author
0,Harry Potter and the Half-Blood Prince (Harry ...,4.56,0439785960,1944099,J.K. Rowling
1,Harry Potter and the Order of the Phoenix (Har...,4.49,0439358078,1996446,J.K. Rowling
2,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,0439554934,5629932,J.K. Rowling
3,Harry Potter and the Chamber of Secrets (Harry...,4.41,0439554896,6267,J.K. Rowling
4,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,043965548X,2149872,J.K. Rowling
5,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,0439682584,38872,J.K. Rowling
6,"Unauthorized Harry Potter Book Seven News: ""Ha...",3.69,0976540606,18,W. Frederick Zimmerman
7,Harry Potter Collection (Harry Potter #1-6),4.73,0439827604,27410,J.K. Rowling
8,The Ultimate Hitchhiker's Guide: Five Complete...,4.38,0517226952,3602,Douglas Adams
9,The Ultimate Hitchhiker's Guide to the Galaxy,4.38,0345453743,240189,Douglas Adams


In [19]:
# new data frame with split value columns - second pass to separate author first and last names
new = books["first_author"].str.split(" ", n = 1, expand = True) 
  
# making separate author columns from new data frame 
books["first_name"]= new[0] 

# making separate author columns from new data frame 
books["last_name"]= new[1] 
  
# Dropping old Name columns 
books.drop(columns =["first_author"], inplace = True) 


# df display 
books


,title,goodreads_rating,isbn10,goodreads_ratings_count,first_name,last_name
0,Harry Potter and the Half-Blood Prince (Harry ...,4.56,0439785960,1944099,J.K.,Rowling
1,Harry Potter and the Order of the Phoenix (Har...,4.49,0439358078,1996446,J.K.,Rowling
2,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,0439554934,5629932,J.K.,Rowling
3,Harry Potter and the Chamber of Secrets (Harry...,4.41,0439554896,6267,J.K.,Rowling
4,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,043965548X,2149872,J.K.,Rowling
5,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,0439682584,38872,J.K.,Rowling
6,"Unauthorized Harry Potter Book Seven News: ""Ha...",3.69,0976540606,18,W.,Frederick Zimmerman
7,Harry Potter Collection (Harry Potter #1-6),4.73,0439827604,27410,J.K.,Rowling
8,The Ultimate Hitchhiker's Guide: Five Complete...,4.38,0517226952,3602,Douglas,Adams
9,The Ultimate Hitchhiker's Guide to the Galaxy,4.38,0345453743,240189,Douglas,Adams


In [20]:
books_ordered = books[['last_name', 'first_name', 'isbn10', 'title', 'goodreads_rating', 'goodreads_ratings_count']]
books_ordered

,last_name,first_name,isbn10,title,goodreads_rating,goodreads_ratings_count
0,Rowling,J.K.,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,4.56,1944099
1,Rowling,J.K.,0439358078,Harry Potter and the Order of the Phoenix (Har...,4.49,1996446
2,Rowling,J.K.,0439554934,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5629932
3,Rowling,J.K.,0439554896,Harry Potter and the Chamber of Secrets (Harry...,4.41,6267
4,Rowling,J.K.,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,2149872
5,Rowling,J.K.,0439682584,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,38872
6,Frederick Zimmerman,W.,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",3.69,18
7,Rowling,J.K.,0439827604,Harry Potter Collection (Harry Potter #1-6),4.73,27410
8,Adams,Douglas,0517226952,The Ultimate Hitchhiker's Guide: Five Complete...,4.38,3602
9,Adams,Douglas,0345453743,The Ultimate Hitchhiker's Guide to the Galaxy,4.38,240189


In [21]:
# new data frame with split value columns - second pass to separate author first and last names
name_fix = books_ordered["last_name"].str.split(n = 1, expand = True) 
name_fix.columns = ['last_name{}'.format(x+1) for x in name_fix.columns]
name_fix2 = books_ordered.join(name_fix)
name_fix2

,last_name,first_name,isbn10,title,goodreads_rating,goodreads_ratings_count,last_name1,last_name2
0,Rowling,J.K.,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,4.56,1944099,Rowling,None
1,Rowling,J.K.,0439358078,Harry Potter and the Order of the Phoenix (Har...,4.49,1996446,Rowling,None
2,Rowling,J.K.,0439554934,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5629932,Rowling,None
3,Rowling,J.K.,0439554896,Harry Potter and the Chamber of Secrets (Harry...,4.41,6267,Rowling,None
4,Rowling,J.K.,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,2149872,Rowling,None
5,Rowling,J.K.,0439682584,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,38872,Rowling,None
6,Frederick Zimmerman,W.,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",3.69,18,Frederick,Zimmerman
7,Rowling,J.K.,0439827604,Harry Potter Collection (Harry Potter #1-6),4.73,27410,Rowling,None
8,Adams,Douglas,0517226952,The Ultimate Hitchhiker's Guide: Five Complete...,4.38,3602,Adams,None
9,Adams,Douglas,0345453743,The Ultimate Hitchhiker's Guide to the Galaxy,4.38,240189,Adams,None


In [22]:
name_fix2.fillna(value="xxx", inplace=True)
name_fix2

,last_name,first_name,isbn10,title,goodreads_rating,goodreads_ratings_count,last_name1,last_name2
0,Rowling,J.K.,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,4.56,1944099,Rowling,xxx
1,Rowling,J.K.,0439358078,Harry Potter and the Order of the Phoenix (Har...,4.49,1996446,Rowling,xxx
2,Rowling,J.K.,0439554934,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5629932,Rowling,xxx
3,Rowling,J.K.,0439554896,Harry Potter and the Chamber of Secrets (Harry...,4.41,6267,Rowling,xxx
4,Rowling,J.K.,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,2149872,Rowling,xxx
5,Rowling,J.K.,0439682584,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,38872,Rowling,xxx
6,Frederick Zimmerman,W.,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",3.69,18,Frederick,Zimmerman
7,Rowling,J.K.,0439827604,Harry Potter Collection (Harry Potter #1-6),4.73,27410,Rowling,xxx
8,Adams,Douglas,0517226952,The Ultimate Hitchhiker's Guide: Five Complete...,4.38,3602,Adams,xxx
9,Adams,Douglas,0345453743,The Ultimate Hitchhiker's Guide to the Galaxy,4.38,240189,Adams,xxx


In [23]:

name_fix2.loc[name_fix2['last_name2'].str.contains('xxx'), 'last_name2'] = name_fix2["last_name1"]
name_fix2.head()

,last_name,first_name,isbn10,title,goodreads_rating,goodreads_ratings_count,last_name1,last_name2
0,Rowling,J.K.,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,4.56,1944099,Rowling,Rowling
1,Rowling,J.K.,0439358078,Harry Potter and the Order of the Phoenix (Har...,4.49,1996446,Rowling,Rowling
2,Rowling,J.K.,0439554934,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5629932,Rowling,Rowling
3,Rowling,J.K.,0439554896,Harry Potter and the Chamber of Secrets (Harry...,4.41,6267,Rowling,Rowling
4,Rowling,J.K.,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,2149872,Rowling,Rowling


In [24]:
name_fix2.drop(columns =["last_name"], inplace = True) 
name_fix2

,first_name,isbn10,title,goodreads_rating,goodreads_ratings_count,last_name1,last_name2
0,J.K.,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,4.56,1944099,Rowling,Rowling
1,J.K.,0439358078,Harry Potter and the Order of the Phoenix (Har...,4.49,1996446,Rowling,Rowling
2,J.K.,0439554934,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5629932,Rowling,Rowling
3,J.K.,0439554896,Harry Potter and the Chamber of Secrets (Harry...,4.41,6267,Rowling,Rowling
4,J.K.,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,2149872,Rowling,Rowling
5,J.K.,0439682584,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,38872,Rowling,Rowling
6,W.,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",3.69,18,Frederick,Zimmerman
7,J.K.,0439827604,Harry Potter Collection (Harry Potter #1-6),4.73,27410,Rowling,Rowling
8,Douglas,0517226952,The Ultimate Hitchhiker's Guide: Five Complete...,4.38,3602,Adams,Adams
9,Douglas,0345453743,The Ultimate Hitchhiker's Guide to the Galaxy,4.38,240189,Adams,Adams


In [25]:
name_fix2.drop(columns =["last_name1"], inplace = True) 
name_fix2

,first_name,isbn10,title,goodreads_rating,goodreads_ratings_count,last_name2
0,J.K.,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,4.56,1944099,Rowling
1,J.K.,0439358078,Harry Potter and the Order of the Phoenix (Har...,4.49,1996446,Rowling
2,J.K.,0439554934,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5629932,Rowling
3,J.K.,0439554896,Harry Potter and the Chamber of Secrets (Harry...,4.41,6267,Rowling
4,J.K.,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,2149872,Rowling
5,J.K.,0439682584,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,38872,Rowling
6,W.,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",3.69,18,Zimmerman
7,J.K.,0439827604,Harry Potter Collection (Harry Potter #1-6),4.73,27410,Rowling
8,Douglas,0517226952,The Ultimate Hitchhiker's Guide: Five Complete...,4.38,3602,Adams
9,Douglas,0345453743,The Ultimate Hitchhiker's Guide to the Galaxy,4.38,240189,Adams


In [26]:
books_cleaned = name_fix2[['last_name2', 'first_name', 'isbn10', 'title', 'goodreads_rating', 'goodreads_ratings_count']]
books_cleaned

,last_name2,first_name,isbn10,title,goodreads_rating,goodreads_ratings_count
0,Rowling,J.K.,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,4.56,1944099
1,Rowling,J.K.,0439358078,Harry Potter and the Order of the Phoenix (Har...,4.49,1996446
2,Rowling,J.K.,0439554934,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5629932
3,Rowling,J.K.,0439554896,Harry Potter and the Chamber of Secrets (Harry...,4.41,6267
4,Rowling,J.K.,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,2149872
5,Rowling,J.K.,0439682584,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,38872
6,Zimmerman,W.,0976540606,"Unauthorized Harry Potter Book Seven News: ""Ha...",3.69,18
7,Rowling,J.K.,0439827604,Harry Potter Collection (Harry Potter #1-6),4.73,27410
8,Adams,Douglas,0517226952,The Ultimate Hitchhiker's Guide: Five Complete...,4.38,3602
9,Adams,Douglas,0345453743,The Ultimate Hitchhiker's Guide to the Galaxy,4.38,240189


In [27]:
books_cleaned = books_cleaned.rename(columns={"last_name2":"last_name"})
books_cleaned.head()


,last_name,first_name,isbn10,title,goodreads_rating,goodreads_ratings_count
0,Rowling,J.K.,0439785960,Harry Potter and the Half-Blood Prince (Harry ...,4.56,1944099
1,Rowling,J.K.,0439358078,Harry Potter and the Order of the Phoenix (Har...,4.49,1996446
2,Rowling,J.K.,0439554934,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5629932
3,Rowling,J.K.,0439554896,Harry Potter and the Chamber of Secrets (Harry...,4.41,6267
4,Rowling,J.K.,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,4.55,2149872


In [28]:
books_cleaned.dtypes

last_name                   object
first_name                  object
isbn10                      object
title                       object
goodreads_rating           float64
goodreads_ratings_count      int64
dtype: object

In [29]:
books_cleaned.to_csv("books_cleaned.csv", index=False)

In [34]:
rds_connection_string = "postgres:********@localhost:5432/Goodreads"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [31]:
engine.table_names()

[]

In [32]:
books_cleaned.to_sql(name='goodreads', con=engine, if_exists='append', index=True)